## Week 05 : 웹 크롤링 II

In [1]:
import re
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime, timedelta

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from collections import Counter
from itertools import chain
from operator import itemgetter
        
basic_header = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)\AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
                "Accept":"text/html,application/xhtml+xml,application/xml;\q=0.9,imgwebp,*/*;q=0.8"}

In [2]:
def url_to_soup(input_url) :
    session = requests.Session()
    req = session.get(input_url, headers=basic_header, allow_redirects=True)
    soup = BeautifulSoup(req.text, 'lxml')
    return soup

soup = url_to_soup('http://underscore.kr/')

In [3]:
for h2 in soup.find_all('h2') :
    print(h2.a['href'])

http://underscore.kr/istrollingcontagious/
http://underscore.kr/politaku/
http://underscore.kr/genderindanger/
http://underscore.kr/goodhigschoolanduniv/
http://underscore.kr/hotweatherkillspeople/
http://underscore.kr/arenoobsmoreconfident/


In [4]:
title_list = [each.a.text for each in soup.find_all('h2')]
suburl_list = [each.a['href'] for each in soup.find_all('h2')]

underscoredf = pd.DataFrame(zip(title_list, suburl_list), columns=['title', 'url'])

underscoredf['len_title'] = underscoredf['title'].str.len()
underscoredf['len_url'] = underscoredf['url'].str.len()
underscoredf['source'] = 'underscore'
underscoredf['digit_len_title'] = underscoredf['len_title'].astype(str).str.len()
underscoredf['doubled_len_title'] = underscoredf['len_title']*2

In [5]:
predatesoup = soup.find_all('span', {'class':'posted-on'})
date_list = []
for each in predatesoup :
    date_list.append(each.find_all('time')[-1]['datetime'])

In [6]:
predatesoup = soup.find_all('span', {'class':'posted-on'})
date_list = [each.find_all('time')[-1]['datetime'] for each in predatesoup]

In [7]:
underscoredf = \
pd.DataFrame(zip(title_list, suburl_list, date_list), columns=['title', 'url', 'time'])
underscoredf

,title,url,time
0,왜 자꾸 어그로를 끄는 거야!,http://underscore.kr/istrollingcontagious/,2018-09-28T19:30:42+00:00
1,페이스북으로 ‘정잘알’이 될 수 있을까?,http://underscore.kr/politaku/,2018-09-16T00:20:45+00:00
2,당신은 남자답지 않습니다,http://underscore.kr/genderindanger/,2018-09-08T23:59:15+00:00
3,좋은 고등학교를 가면 대학 입시에서 유리할까?,http://underscore.kr/goodhigschoolanduniv/,2018-08-28T20:01:48+00:00
4,"덥다, 더워! 데이터가 알려주는 더위와 자살률의 관계",http://underscore.kr/hotweatherkillspeople/,2018-08-13T20:53:25+00:00
5,진짜로 잘 모르는 사람이 더 용감할까?,http://underscore.kr/arenoobsmoreconfident/,2018-08-02T18:18:07+00:00


In [8]:
underscoredf['url']

0     http://underscore.kr/istrollingcontagious/
1                 http://underscore.kr/politaku/
2           http://underscore.kr/genderindanger/
3     http://underscore.kr/goodhigschoolanduniv/
4    http://underscore.kr/hotweatherkillspeople/
5    http://underscore.kr/arenoobsmoreconfident/
Name: url, dtype: object

In [ ]:
tag_list = []
for each_url in tqdm(underscoredf['url']) :
    time.sleep(1)
    cnsoup = url_to_soup(each_url)
    tag_list.append([a.text for a in cnsoup.find('span', {'class':'tags-links'}).find_all('a')])

 67%|████████████████████████████████████████████████████████                            | 4/6 [00:05<00:02,  1.35s/it]

In [ ]:
mainsoup = url_to_soup('https://imnews.imbc.com/pc_main.html')

mbc_title_list = []
mbc_url_list = []
mbc_writer_list = []

for li in mainsoup.find_all('li', {'class':'ellipsis'}) :
    mbc_url_list.append(li.a['href'])
    mbc_title_list.append(li.a.text)
    
for article_url in mbc_url_list :
    time.sleep(.7)
    article_soup = url_to_soup(article_url)
    mbc_writer_list.append(article_soup.find('span', {'class':'writer'}).text)  

In [ ]:
pd.DataFrame(zip(mbc_title_list, mbc_url_list, mbc_writer_list))